<a href="https://colab.research.google.com/github/mahimscit/DO180-apps/blob/master/SQLCaseStudy_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SQL Case Study - 1

In [1]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE FactTable (
    Date TEXT,
    ProductID INTEGER,
    Profit REAL,
    Sales REAL,
    Margin REAL,
    COGS REAL,
    TotalExpenses REAL,
    Marketing REAL,
    Inventory INTEGER,
    BudgetProfit REAL,
    BudgetCOGS REAL,
    BudgetMargin REAL,
    BudgetSales REAL,
    AreaCode INTEGER
);
""")

cursor.execute("""
CREATE TABLE ProductTable (
    ProductType TEXT,
    Product TEXT,
    ProductID INTEGER,
    Type TEXT
);
""")

cursor.execute("""
CREATE TABLE LocationTable (
    AreaCode INTEGER,
    State TEXT,
    Market TEXT,
    MarketSize TEXT
);
""")

cursor.execute("INSERT INTO FactTable VALUES ('2023-11-15', 1, 10.50, 50.00, 0.21, 39.50, 5.00, 2.00, 100, 12.00, 38.00, 0.24, 52.00, 10001);")

cursor.execute("INSERT INTO ProductTable VALUES ('Electronics', 'Smartphone', 1, 'Gadget');")

cursor.execute("INSERT INTO LocationTable VALUES (10001, 'California', 'West Coast', 'Large');")

cursor.execute("SELECT * FROM FactTable;")
print(cursor.fetchall())

cursor.execute("SELECT * FROM ProductTable;")
print(cursor.fetchall())

cursor.execute("SELECT * FROM LocationTable;")
print(cursor.fetchall())

conn.commit()
conn.close()

[('2023-11-15', 1, 10.5, 50.0, 0.21, 39.5, 5.0, 2.0, 100, 12.0, 38.0, 0.24, 52.0, 10001)]
[('Electronics', 'Smartphone', 1, 'Gadget')]
[(10001, 'California', 'West Coast', 'Large')]


In [2]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

# 1. Sales and Profit by State
cursor.execute("""
SELECT l.State, SUM(f.Sales) AS TotalSales, SUM(f.Profit) AS TotalProfit
FROM FactTable f
JOIN LocationTable l ON f.AreaCode = l.AreaCode
GROUP BY l.State
ORDER BY TotalSales DESC;
""")
sales_profit_by_state = cursor.fetchall()

# 2. Marketing Expenses by State
cursor.execute("""
SELECT l.State, SUM(f.Marketing) AS TotalMarketingExpenses
FROM FactTable f
JOIN LocationTable l ON f.AreaCode = l.AreaCode
GROUP BY l.State
ORDER BY TotalMarketingExpenses DESC;
""")
marketing_expenses_by_state = cursor.fetchall()

# 3. COGS, Budget Profit, and Margin by Product
cursor.execute("""
SELECT p.Product, SUM(f.COGS) AS TotalCOGS, SUM(f.BudgetProfit) AS TotalBudgetProfit, AVG(f.Margin) AS AverageMargin
FROM FactTable f
JOIN ProductTable p ON f.ProductID = p.ProductID
GROUP BY p.Product
ORDER BY TotalCOGS DESC;
""")
cogs_budget_margin_by_product = cursor.fetchall()

# 4. Best-Selling Items (by Sales)
cursor.execute("""
SELECT p.Product, SUM(f.Sales) AS TotalSales
FROM FactTable f
JOIN ProductTable p ON f.ProductID = p.ProductID
GROUP BY p.Product
ORDER BY TotalSales DESC
LIMIT 10;
""")
best_selling_items = cursor.fetchall()

# 5. Items with Highest Profit Margin
cursor.execute("""
SELECT p.Product, AVG(f.Margin) AS AverageMargin
FROM FactTable f
JOIN ProductTable p ON f.ProductID = p.ProductID
GROUP BY p.Product
ORDER BY AverageMargin DESC
LIMIT 10;
""")
highest_profit_margin_items = cursor.fetchall()

print("1. Sales and Profit by State:")
for row in sales_profit_by_state:
    print(f"  State: {row[0]}, Total Sales: {row[1]}, Total Profit: {row[2]}")

print("\n2. Marketing Expenses by State:")
for row in marketing_expenses_by_state:
    print(f"  State: {row[0]}, Total Marketing Expenses: {row[1]}")

print("\n3. COGS, Budget Profit, and Margin by Product:")
for row in cogs_budget_margin_by_product:
    print(f"  Product: {row[0]}, Total COGS: {row[1]}, Total Budget Profit: {row[2]}, Average Margin: {row[3]}")

print("\n4. Best-Selling Items (by Sales):")
for row in best_selling_items:
    print(f"  Product: {row[0]}, Total Sales: {row[1]}")

print("\n5. Items with Highest Profit Margin:")
for row in highest_profit_margin_items:
    print(f"  Product: {row[0]}, Average Margin: {row[1]}")

conn.close()

1. Sales and Profit by State:
  State: California, Total Sales: 50.0, Total Profit: 10.5

2. Marketing Expenses by State:
  State: California, Total Marketing Expenses: 2.0

3. COGS, Budget Profit, and Margin by Product:
  Product: Smartphone, Total COGS: 39.5, Total Budget Profit: 12.0, Average Margin: 0.21

4. Best-Selling Items (by Sales):
  Product: Smartphone, Total Sales: 50.0

5. Items with Highest Profit Margin:
  Product: Smartphone, Average Margin: 0.21


In [3]:
import sqlite3
import random
import datetime

def populate_fact_table(db_file, num_records=4200):
    """Populates the FactTable with random data."""
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    try:
        product_ids = [1, 2, 3, 4, 5]
        states = ["California", "Texas", "New York", "Florida", "Illinois"]
        area_codes = [10001, 10002, 10003, 10004, 10005]

        for i in range(num_records):
            date = datetime.date(2023, random.randint(1, 12), random.randint(1, 28))
            product_id = random.choice(product_ids)
            profit = random.uniform(0, 100)
            sales = random.uniform(0, 1000)
            margin = profit / sales if sales else 0
            cogs = sales - profit
            total_expenses = random.uniform(0, 100)
            marketing = random.uniform(0, 100)
            inventory = random.randint(0, 100)
            budget_profit = random.uniform(0, 100)
            budget_cogs = random.uniform(0, 1000)
            budget_margin = budget_profit / budget_cogs if budget_cogs else 0
            budget_sales = budget_cogs + budget_profit
            area_code = random.choice(area_codes)

            cursor.execute("""
                INSERT INTO FactTable (Date, ProductID, Profit, Sales, Margin, COGS, TotalExpenses, Marketing, Inventory, BudgetProfit, BudgetCOGS, BudgetMargin, BudgetSales, AreaCode)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
            """, (date, product_id, profit, sales, margin, cogs, total_expenses, marketing, inventory, budget_profit, budget_cogs, budget_margin, budget_sales, area_code))

        conn.commit()
        print(f"{num_records} records inserted into FactTable successfully!")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    finally:
        conn.close()

database_file = 'customer_data.db'
populate_fact_table(database_file)

4200 records inserted into FactTable successfully!


In [4]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.executemany("INSERT INTO ProductTable VALUES (?, ?, ?, ?)", [
    ('Electronics', 'Smartphone', 1, 'Gadget'),
    ('Electronics', 'Laptop', 2, 'Computer'),
    ('Electronics', 'Tablet', 3, 'Gadget'),
    ('Clothing', 'T-shirt', 4, 'Apparel'),
    ('Clothing', 'Jeans', 5, 'Apparel'),
    ('Clothing', 'Dress', 6, 'Apparel'),
    ('Furniture', 'Sofa', 7, 'Living Room'),
    ('Furniture', 'Bed', 8, 'Bedroom'),
    ('Furniture', 'Table', 9, 'Dining Room'),
    ('Books', 'Novel', 10, 'Fiction'),
    ('Books', 'Textbook', 11, 'Non-fiction'),
    ('Books', 'Comic Book', 12, 'Fiction'),
    ('Kitchenware', 'Knife Set', 13, 'Utensils')

])

conn.commit()
conn.close()

In [6]:
import sqlite3
import random

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

states = ["California", "Texas", "New York", "Florida", "Illinois", "Pennsylvania", "Ohio", "Michigan", "Georgia", "North Carolina", "New Jersey", "Virginia", "Washington", "Arizona", "Massachusetts", "Tennessee", "Indiana", "Missouri", "Maryland", "Wisconsin", "Colorado", "Minnesota", "South Carolina", "Alabama", "Louisiana", "Kentucky", "Oregon", "Oklahoma", "Connecticut", "Utah", "Nevada", "Iowa", "Arkansas", "Mississippi", "Kansas", "New Mexico", "Nebraska", "West Virginia", "Idaho", "Hawaii", "New Hampshire", "Maine", "Rhode Island", "Montana", "Delaware", "South Dakota", "North Dakota", "Alaska", "District of Columbia", "Vermont", "Wyoming"]
markets = ["West Coast", "East Coast", "Midwest", "South", "Southwest", "Northeast", "Southeast", "Northwest", "Central"]
market_sizes = ["Large", "Medium", "Small"]

for area_code in range(10001, 10157):
    state = random.choice(states)
    market = random.choice(markets)
    market_size = random.choice(market_sizes)
    cursor.execute("INSERT INTO LocationTable VALUES (?, ?, ?, ?)", (area_code, state, market, market_size))

conn.commit()
conn.close()

1.Display the number of states present in the LocationTable.

In [7]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("SELECT COUNT(DISTINCT State) FROM LocationTable;")
num_states = cursor.fetchone()[0]

print(f"Number of states in LocationTable: {num_states}")

conn.close()

Number of states in LocationTable: 51


2. How many products are of regular type?

In [9]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.executemany("INSERT INTO ProductTable VALUES (?, ?, ?, ?)", [
    ('Electronics', 'Regular TV', 14, 'Regular'),
    ('Clothing', 'Regular Shirt', 15, 'Regular'),
    ('Furniture', 'Regular Chair', 16, 'Regular'),
])

conn.commit()
conn.close()

In [10]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM ProductTable WHERE Type = 'Regular';")
num_regular_products = cursor.fetchone()[0]

print(f"Number of regular products: {num_regular_products}")

conn.close()

Number of regular products: 3


3. How much spending has been done on marketing of product ID 1?

In [11]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("SELECT SUM(Marketing) FROM FactTable WHERE ProductID = 1;")
total_marketing_spending = cursor.fetchone()[0]

print(f"Total marketing spending for product ID 1: {total_marketing_spending}")

conn.close()

Total marketing spending for product ID 1: 42912.801326353394


4. What is the minimum sales of a product?

In [12]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("SELECT MIN(Sales) FROM FactTable;")
min_sales = cursor.fetchone()[0]

print(f"Minimum sales of a product: {min_sales}")

conn.close()

Minimum sales of a product: 0.046760212433105686


5. Display the max Cost of Good Sold (COGS).

In [13]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("SELECT MAX(COGS) FROM FactTable;")
max_cogs = cursor.fetchone()[0]

print(f"Maximum Cost of Goods Sold (COGS): {max_cogs}")

conn.close()

Maximum Cost of Goods Sold (COGS): 994.954027687449


6. Display the details of the product where product type is coffee.

In [14]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM ProductTable WHERE ProductType = 'Coffee';")
coffee_products = cursor.fetchall()

for product in coffee_products:
    print(product)

conn.close()

7. Display the details where total expenses are greater than 40

In [15]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM FactTable WHERE TotalExpenses > 40;")
high_expense_records = cursor.fetchall()

for record in high_expense_records:
    print(record)

conn.close()

('2023-11-13', 3, 8.845767309651542, 405.48253378608115, 0.021815408982124664, 396.6367664764296, 89.58572951544758, 48.51174219680763, 13, 35.54747617279118, 438.37484384707926, 0.08108922460247606, 473.9223200198704, 10002)
('2023-01-12', 3, 31.463139638461058, 509.4306022347284, 0.06176138516304505, 477.9674625962673, 57.44333043931786, 80.51396241295936, 36, 87.1296918180506, 709.3502168467841, 0.12283028854965468, 796.4799086648346, 10004)
('2023-03-15', 2, 54.15498209526049, 18.857654345399546, 2.8717772159437196, -35.29732774986095, 46.73784223315498, 40.61767849172237, 33, 27.356059444523005, 206.25800905227098, 0.13263028946231267, 233.61406849679398, 10002)
('2023-03-11', 2, 28.435662668496576, 179.91643004328907, 0.158049282445493, 151.4807673747925, 56.54588673230416, 72.7283487073543, 44, 58.509970206277906, 550.1936582435942, 0.10634431954934138, 608.7036284498721, 10003)
('2023-07-08', 4, 5.606677490777656, 338.4274718620155, 0.016566850970844427, 332.82079437123787, 47.

8. What is the average sales in area code 719?

In [16]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("SELECT AVG(Sales) FROM FactTable WHERE AreaCode = 719;")
avg_sales_719 = cursor.fetchone()[0]

print(f"Average sales in area code 719: {avg_sales_719}")

conn.close()

Average sales in area code 719: None


9. Find out the total profit generated by Colorado state.

In [17]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT SUM(f.Profit) AS TotalProfit
FROM FactTable f
JOIN LocationTable l ON f.AreaCode = l.AreaCode
WHERE l.State = 'Colorado';
""")
total_profit_colorado = cursor.fetchone()[0]

print(f"Total profit generated by Colorado: {total_profit_colorado}")

conn.close()

Total profit generated by Colorado: None


10. Display the average inventory for each product ID.

In [18]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT ProductID, AVG(Inventory) AS AverageInventory
FROM FactTable
GROUP BY ProductID;
""")
average_inventory_by_product = cursor.fetchall()

for product_id, avg_inventory in average_inventory_by_product:
    print(f"Product ID: {product_id}, Average Inventory: {avg_inventory}")

conn.close()

Product ID: 1, Average Inventory: 51.303899082568805
Product ID: 2, Average Inventory: 48.187344913151364
Product ID: 3, Average Inventory: 50.14982164090369
Product ID: 4, Average Inventory: 51.48946135831382
Product ID: 5, Average Inventory: 49.83816425120773


11. Display state in a sequential order in a Location Table.

In [20]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("SELECT distinct(State) FROM LocationTable ORDER BY State ASC;")
states_in_order = cursor.fetchall()

for state in states_in_order:
    print(state[0])
conn.close()

Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming


12. Display the average budget of the Product where the average budget
margin should be greater than 100.

In [21]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT ProductID, AVG(BudgetSales) AS AverageBudget
FROM FactTable
GROUP BY ProductID
HAVING AVG(BudgetMargin) > 100;
""")
results = cursor.fetchall()

for product_id, avg_budget in results:
    print(f"Product ID: {product_id}, Average Budget: {avg_budget}")

conn.close()

13. What is the total sales done on date 2010-01-01?

In [22]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT SUM(Sales) AS TotalSales
FROM FactTable
WHERE Date = '2010-01-01';
""")
total_sales_20100101 = cursor.fetchone()[0]

print(f"Total sales on 2010-01-01: {total_sales_20100101}")

conn.close()

Total sales on 2010-01-01: None


14. Display the average total expense of each product ID on an individual date.

In [24]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT distinct(ProductID), Date, AVG(TotalExpenses) AS AverageTotalExpense
FROM FactTable
GROUP BY ProductID, Date;
""")
results = cursor.fetchall()

for product_id, date, avg_total_expense in results:
    print(f"Product ID: {product_id}, Date: {date}, Average Total Expense: {avg_total_expense}")

conn.close()

Product ID: 1, Date: 2023-01-02, Average Total Expense: 74.68261670538946
Product ID: 1, Date: 2023-01-03, Average Total Expense: 62.244507758575715
Product ID: 1, Date: 2023-01-04, Average Total Expense: 96.81015545408178
Product ID: 1, Date: 2023-01-05, Average Total Expense: 98.72121882133533
Product ID: 1, Date: 2023-01-06, Average Total Expense: 22.025588580457537
Product ID: 1, Date: 2023-01-07, Average Total Expense: 45.00251215492486
Product ID: 1, Date: 2023-01-08, Average Total Expense: 48.7472934200209
Product ID: 1, Date: 2023-01-09, Average Total Expense: 46.927508046432706
Product ID: 1, Date: 2023-01-10, Average Total Expense: 0.7440447339115308
Product ID: 1, Date: 2023-01-11, Average Total Expense: 48.902342153460836
Product ID: 1, Date: 2023-01-12, Average Total Expense: 30.776268096489968
Product ID: 1, Date: 2023-01-13, Average Total Expense: 45.66091416361801
Product ID: 1, Date: 2023-01-14, Average Total Expense: 41.15823494269234
Product ID: 1, Date: 2023-01-16, 

15. Display the table with the following attributes such as date, product ID,
product_type, product, sales, profit, state, area_code.

In [25]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT
    f.Date,
    f.ProductID,
    p.ProductType,
    p.Product,
    f.Sales,
    f.Profit,
    l.State,
    f.AreaCode
FROM FactTable f
JOIN ProductTable p ON f.ProductID = p.ProductID
JOIN LocationTable l ON f.AreaCode = l.AreaCode;
""")
results = cursor.fetchall()

for row in results:
    print(row)

conn.close()

Streaming output truncated to the last 5000 lines.
('2023-12-07', 2, 'Electronics', 'Laptop', 214.84488550826285, 62.12252932750799, 'Delaware', 10001)
('2023-12-07', 2, 'Electronics', 'Laptop', 214.84488550826285, 62.12252932750799, 'North Dakota', 10001)
('2023-11-05', 3, 'Electronics', 'Tablet', 404.8859076283502, 55.81993779432557, 'California', 10001)
('2023-11-05', 3, 'Electronics', 'Tablet', 404.8859076283502, 55.81993779432557, 'Delaware', 10001)
('2023-11-05', 3, 'Electronics', 'Tablet', 404.8859076283502, 55.81993779432557, 'North Dakota', 10001)
('2023-01-17', 1, 'Electronics', 'Smartphone', 936.887809011973, 69.91818063936203, 'California', 10005)
('2023-01-17', 1, 'Electronics', 'Smartphone', 936.887809011973, 69.91818063936203, 'Hawaii', 10005)
('2023-01-17', 1, 'Electronics', 'Smartphone', 936.887809011973, 69.91818063936203, 'California', 10005)
('2023-01-17', 1, 'Electronics', 'Smartphone', 936.887809011973, 69.91818063936203, 'Hawaii', 10005)
('2023-12-25', 1, 'Electr

16. Display the rank without any gap to show the sales wise rank.

In [26]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT
    Date,
    ProductID,
    Sales,
    DENSE_RANK() OVER (ORDER BY Sales DESC) AS SalesRank
FROM FactTable;
""")
results = cursor.fetchall()


for row in results:
    print(row)

conn.close()

('2023-05-22', 5, 999.6954562293074, 1)
('2023-11-17', 4, 999.5118161648568, 2)
('2023-05-14', 1, 999.4945188645924, 3)
('2023-04-15', 3, 999.4366449234067, 4)
('2023-05-18', 1, 999.2309575582534, 5)
('2023-01-26', 5, 998.8749121736456, 6)
('2023-04-04', 5, 998.7455946646866, 7)
('2023-04-03', 1, 998.7199133352362, 8)
('2023-02-21', 1, 998.3587691416724, 9)
('2023-08-23', 4, 997.5999208590641, 10)
('2023-04-08', 1, 997.2898373540291, 11)
('2023-03-11', 3, 997.2440353645884, 12)
('2023-09-24', 1, 996.9786763364909, 13)
('2023-12-08', 3, 996.9307212234852, 14)
('2023-07-21', 3, 996.8675710657819, 15)
('2023-11-27', 4, 996.520749784285, 16)
('2023-06-21', 1, 996.33849912442, 17)
('2023-09-14', 5, 996.2725381535099, 18)
('2023-09-04', 4, 996.0295320286017, 19)
('2023-04-27', 4, 995.9976181538317, 20)
('2023-08-28', 5, 995.4124092138762, 21)
('2023-02-11', 3, 995.3164518831443, 22)
('2023-04-06', 1, 995.3049637708425, 23)
('2023-11-09', 4, 994.9735907807699, 24)
('2023-03-26', 5, 994.852750

17. Find the state wise profit and sales.

In [27]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT
    l.State,
    SUM(f.Sales) AS TotalSales,
    SUM(f.Profit) AS TotalProfit
FROM FactTable f
JOIN LocationTable l ON f.AreaCode = l.AreaCode
GROUP BY l.State
ORDER BY l.State;
""")
results = cursor.fetchall()


for row in results:
    print(row)

conn.close()

('California', 833212.2656627278, 81850.78571954762)
('Delaware', 400528.4030922035, 38645.00832972607)
('Hawaii', 878653.512349029, 87503.78334760165)
('Indiana', 428243.9005328302, 44187.93406336712)
('Iowa', 386075.6502951001, 39636.34309049302)
('North Dakota', 400528.4030922035, 38645.00832972607)
('Pennsylvania', 445969.64977850433, 44298.005957780144)
('Washington', 386075.6502951001, 39636.34309049302)
('Wyoming', 428243.9005328302, 44187.93406336712)


18. Find the state wise profit and sales along with the product name.

In [28]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT
    l.State,
    p.Product,
    SUM(f.Sales) AS TotalSales,
    SUM(f.Profit) AS TotalProfit
FROM FactTable f
JOIN LocationTable l ON f.AreaCode = l.AreaCode
JOIN ProductTable p ON f.ProductID = p.ProductID
GROUP BY l.State, p.Product
ORDER BY l.State, p.Product;
""")
results = cursor.fetchall()

for row in results:
    print(row)

conn.close()

('California', 'Jeans', 159814.99023509878, 15942.409803186489)
('California', 'Laptop', 164197.25431509802, 16171.891538035361)
('California', 'Smartphone', 382849.9681597665, 33768.67500430371)
('California', 'T-shirt', 146095.52702483107, 15953.04408905408)
('California', 'Tablet', 171679.5100078161, 16899.102787119926)
('Delaware', 'Jeans', 77432.38151410312, 7641.222071680825)
('Delaware', 'Laptop', 82739.58446627484, 7649.772678132642)
('Delaware', 'Smartphone', 169524.93738835325, 15298.674085197406)
('Delaware', 'T-shirt', 69080.67157977782, 7297.033821304885)
('Delaware', 'Tablet', 86513.29683787108, 8407.642716009019)
('Hawaii', 'Jeans', 169420.8969927193, 17210.874767638048)
('Hawaii', 'Laptop', 160702.217725716, 16162.204320081233)
('Hawaii', 'Smartphone', 393389.53923785547, 35557.64349519329)
('Hawaii', 'T-shirt', 175900.99211651002, 18326.72363962768)
('Hawaii', 'Tablet', 175934.63589515508, 18025.158872658216)
('Indiana', 'Jeans', 77623.67853242616, 7312.6532725382485)


19. If there is an increase in sales of 5%, calculate the increasedsales.

In [29]:
import sqlite3

conn = sqlite3.connect('customer_data.db')

cursor = conn.cursor()

cursor.execute("""
SELECT
    ProductID,
    Sales,
    Sales * 1.05 AS IncreasedSales
FROM FactTable;
""")
results = cursor.fetchall()

for row in results:
    print(row)

conn.close()

(1, 50.0, 52.5)
(3, 405.48253378608115, 425.7566604753852)
(2, 850.1592585877333, 892.6672215171201)
(3, 509.4306022347284, 534.9021323464648)
(2, 3.2289510134787536, 3.3903985641526915)
(1, 105.4337086274052, 110.70539405877545)
(2, 18.857654345399546, 19.800537062669523)
(2, 179.91643004328907, 188.91225154545353)
(4, 338.4274718620155, 355.3488454551163)
(3, 928.6303608579067, 975.0618789008021)
(1, 90.47928335593103, 95.00324752372758)
(4, 369.44105509185187, 387.91310784644446)
(1, 777.3130035441619, 816.1786537213701)
(2, 320.59357333482075, 336.6232520015618)
(2, 623.061990270546, 654.2150897840733)
(5, 235.46899799961963, 247.24244789960062)
(3, 545.4746309106441, 572.7483624561763)
(3, 163.51278414752736, 171.68842335490373)
(1, 416.06110860551837, 436.8641640357943)
(3, 252.78999706788176, 265.42949692127587)
(2, 870.3005802887996, 913.8156093032395)
(1, 579.0491749164177, 608.0016336622385)
(1, 71.3155758187658, 74.88135460970409)
(4, 876.9512946463196, 920.7988593786357)
(4

20. Find the maximum profit along with the product ID and producttype.

In [30]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT
    p.ProductID,
    p.ProductType,
    MAX(f.Profit) AS MaximumProfit
FROM FactTable f
JOIN ProductTable p ON f.ProductID = p.ProductID
GROUP BY p.ProductID, p.ProductType
ORDER BY MaximumProfit DESC
LIMIT 1;
""")
result = cursor.fetchone()

print(f"Product ID: {result[0]}, Product Type: {result[1]}, Maximum Profit: {result[2]}")

conn.close()

Product ID: 5, Product Type: Clothing, Maximum Profit: 99.99039684783048


21. Create a stored procedure to fetch the result according to the product type from Product Table.

In [33]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

def GetProductsByType(productType):
    cursor.execute("SELECT * FROM ProductTable WHERE ProductType=?", (productType,))
    return cursor.fetchall()

results = GetProductsByType('Electronics')

for row in results:
    print(row)

conn.close()

('Electronics', 'Smartphone', 1, 'Gadget')
('Electronics', 'Smartphone', 1, 'Gadget')
('Electronics', 'Laptop', 2, 'Computer')
('Electronics', 'Tablet', 3, 'Gadget')
('Electronics', 'Regular TV', 14, 'Regular')


In [ ]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
CREATE PROCEDURE GetProductsByType(productType TEXT)
AS
BEGIN
  SELECT * FROM ProductTable WHERE ProductType = productType;
END;
""")

cursor.execute("SELECT * FROM GetProductsByType('Electronics');")
results = cursor.fetchall()

for row in results:
    print(row)

conn.close()

22. Write a query by creating a condition in which if the total expenses is less than 60 then it is a profit or else loss.

In [35]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT
    ProductID,
    Date,
    TotalExpenses,
    CASE
        WHEN TotalExpenses < 60 THEN 'Profit'
        ELSE 'Loss'
    END AS ProfitOrLoss
FROM FactTable;
""")
results = cursor.fetchall()


for row in results:
    print(row)

conn.close()

(1, '2023-11-15', 5.0, 'Profit')
(3, '2023-11-13', 89.58572951544758, 'Loss')
(2, '2023-12-06', 28.264525527613316, 'Profit')
(3, '2023-01-12', 57.44333043931786, 'Profit')
(2, '2023-09-18', 16.669685610068708, 'Profit')
(1, '2023-08-11', 6.287682241134562, 'Profit')
(2, '2023-03-15', 46.73784223315498, 'Profit')
(2, '2023-03-11', 56.54588673230416, 'Profit')
(4, '2023-07-08', 47.54232463300799, 'Profit')
(3, '2023-03-17', 27.21086241765719, 'Profit')
(1, '2023-09-12', 67.20014628002843, 'Loss')
(4, '2023-01-23', 84.87719950552942, 'Loss')
(1, '2023-04-07', 81.07094363093798, 'Loss')
(2, '2023-01-19', 9.659847448031655, 'Profit')
(2, '2023-06-02', 23.71795191311413, 'Profit')
(5, '2023-11-06', 73.08684691742103, 'Loss')
(3, '2023-01-05', 63.825232588862626, 'Loss')
(3, '2023-02-20', 21.614019403246488, 'Profit')
(1, '2023-04-12', 17.868968023109645, 'Profit')
(3, '2023-10-02', 34.45336667532386, 'Profit')
(2, '2023-09-01', 57.18074068275503, 'Profit')
(1, '2023-03-23', 24.2023691097587

23. Give the total weekly sales value with the date and product ID details. Use roll-up to pull the data in hierarchical order.

In [37]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

query = """
SELECT
    Date,
    ProductID,
    SUM(Sales) AS WeeklySales
FROM FactTable
GROUP BY Date, ProductID

UNION ALL

SELECT
    Date,
    NULL AS ProductID,  -- Roll-up to weekly level
    SUM(Sales) AS WeeklySales
FROM FactTable
GROUP BY Date

UNION ALL

SELECT
    NULL AS Date,       -- Roll-up to total level
    NULL AS ProductID,
    SUM(Sales) AS WeeklySales
FROM FactTable

ORDER BY Date, ProductID;
"""
cursor.execute(query)
results = cursor.fetchall()

df = pd.DataFrame(results, columns=['Date', 'ProductID', 'WeeklySales'])
print(df)

conn.close()

            Date  ProductID   WeeklySales
0           None        NaN  2.093501e+06
1     2023-01-01        NaN  5.426785e+03
2     2023-01-01        2.0  4.646075e+02
3     2023-01-01        3.0  2.108376e+03
4     2023-01-01        4.0  2.159414e+03
...          ...        ...           ...
1863  2023-12-28        1.0  8.854582e+02
1864  2023-12-28        2.0  1.703545e+03
1865  2023-12-28        3.0  6.618687e+02
1866  2023-12-28        4.0  9.807867e+02
1867  2023-12-28        5.0  1.390342e+03

[1868 rows x 3 columns]


24. Apply union and intersection operator on the tables which consist of
attribute area code.

In [38]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()


cursor.execute("""
SELECT AreaCode FROM FactTable
UNION
SELECT AreaCode FROM LocationTable;
""")
union_results = cursor.fetchall()
print("UNION results:", union_results)


cursor.execute("""
SELECT AreaCode FROM FactTable
INTERSECT
SELECT AreaCode FROM LocationTable;
""")
intersect_results = cursor.fetchall()
print("INTERSECT results:", intersect_results)

conn.close()

UNION results: [(10001,), (10002,), (10003,), (10004,), (10005,), (10006,), (10007,), (10008,), (10009,), (10010,), (10011,), (10012,), (10013,), (10014,), (10015,), (10016,), (10017,), (10018,), (10019,), (10020,), (10021,), (10022,), (10023,), (10024,), (10025,), (10026,), (10027,), (10028,), (10029,), (10030,), (10031,), (10032,), (10033,), (10034,), (10035,), (10036,), (10037,), (10038,), (10039,), (10040,), (10041,), (10042,), (10043,), (10044,), (10045,), (10046,), (10047,), (10048,), (10049,), (10050,), (10051,), (10052,), (10053,), (10054,), (10055,), (10056,), (10057,), (10058,), (10059,), (10060,), (10061,), (10062,), (10063,), (10064,), (10065,), (10066,), (10067,), (10068,), (10069,), (10070,), (10071,), (10072,), (10073,), (10074,), (10075,), (10076,), (10077,), (10078,), (10079,), (10080,), (10081,), (10082,), (10083,), (10084,), (10085,), (10086,), (10087,), (10088,), (10089,), (10090,), (10091,), (10092,), (10093,), (10094,), (10095,), (10096,), (10097,), (10098,), (100

25. Create a user-defined function for the product table to fetch a particular
product type based upon the user’s preference.

In [39]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()


cursor.execute("""
CREATE VIEW ProductsByType AS
SELECT *
FROM ProductTable
WHERE ProductType = 'Electronics';  -- Replace 'Electronics' with the desired product type
""")


cursor.execute("SELECT * FROM ProductsByType;")
results = cursor.fetchall()


for row in results:
    print(row)

conn.close()

('Electronics', 'Smartphone', 1, 'Gadget')
('Electronics', 'Smartphone', 1, 'Gadget')
('Electronics', 'Laptop', 2, 'Computer')
('Electronics', 'Tablet', 3, 'Gadget')
('Electronics', 'Regular TV', 14, 'Regular')


26. Change the product type from coffee to tea where product ID is 1 and undo
it.

In [40]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()


cursor.execute("UPDATE ProductTable SET ProductType = 'Tea' WHERE ProductID = 1;")
conn.commit()


cursor.execute("UPDATE ProductTable SET ProductType = 'Coffee' WHERE ProductID = 1;")
conn.commit()

conn.close()

27. Display the date, product ID and sales where total expenses are
between 100 to 200.

In [41]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("""
SELECT Date, ProductID, Sales
FROM FactTable
WHERE TotalExpenses BETWEEN 100 AND 200;
""")
results = cursor.fetchall()


for row in results:
    print(row)

conn.close()

28. Delete the records in the Product Table for regular type.

In [42]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("DELETE FROM ProductTable WHERE ProductType = 'Regular';")
conn.commit()

conn.close()

29. Display the ASCII value of the fifth character from the columnProduct.

In [46]:
import sqlite3

conn = sqlite3.connect('customer_data.db')
cursor = conn.cursor()

cursor.execute("INSERT INTO ProductTable (ProductType, Product, ProductID, Type) VALUES ('Electronics', 'Phones', 17, 'Gadget');")
conn.commit()

cursor.execute("""
SELECT UNICODE(SUBSTR(Product, 5, 1)) AS FifthCharAscii  -- Change ASCII to UNICODE
FROM ProductTable
WHERE Product = 'Phones';
""")
results = cursor.fetchall()

for row in results:
    print(f"ASCII value of the fifth character: {row[0]}")

conn.close()

ASCII value of the fifth character: 101
ASCII value of the fifth character: 101
ASCII value of the fifth character: 101
